In [13]:
%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Definición de Librería

In [2]:
import tweepy
import json
import pandas as pd
import time


In [3]:
pd.set_option('display.max_rows', 1000)

In [4]:
#Definción de variables
dataFollowing_id = pd.DataFrame()
dataFollowers_id = pd.DataFrame()

# Definición de las claves para acceder a Twitter

In [5]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [6]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)

In [16]:
#Búsqueda de Tweets por palabra
def busqueda_tweets_palabra(palabra="Ryanair", lenguaje="es", numero_de_Tweets=200):
    return tweepy.Cursor(api.search, palabra, lang=lenguaje).items(numero_de_Tweets)

tweets=busqueda_tweets_palabra()
outtweets = [(tweet.created_at, 
              tweet.text, 
              tweet.retweet_count, 
              str(tweet.favorite_count)+'') for tweet in tweets]
outtweets

[(datetime.datetime(2019, 12, 18, 19, 3, 53),
  'Ryanair atribuye los recortes en Girona a los problemas del Boeing 737 MAX https://t.co/asx2LZmyV3 vía @elpais_economia',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 19, 0, 8),
  'Los asientos separados, el nuevo mercadillo de las low cost (Ryanair, Vueling, etc...) https://t.co/Zg75Rhm4re',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 49, 18),
  'RT @marcosleyley: He visto sardinas en lata que tienen más espacio que volar en Vueling o Ryanair. Es curioso que mientras la población mun…',
  3,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 48, 17),
  'si tienes cualquier problema para que puedas resolverlo "perfectamente" te ponen un número de atención al cliente d… https://t.co/e94lP2nyZ5',
  0,
  '1'),
 (datetime.datetime(2019, 12, 18, 18, 40),
  'viajar con @Ryanair_ES y otras maneras de solo tener problemas.',
  0,
  '2'),
 (datetime.datetime(2019, 12, 18, 18, 36, 2),
  'Inspección de Trabajo abrirá expediente ✈️ https://t.co/

# Extracción de los datos de la cuenta

In [42]:
item = api.get_user('@hipercor')
data_account=[]
account={}
account["name"]=item.name
account["id"]=item.id
account["screen name"]=item.screen_name
account["friends_count"]=item.friends_count
account["followers_count"]=item.followers_count
data_account.append(account)
df_account_info = pd.DataFrame(data_account)
df_account_info

,name,id,screen name,friends_count,followers_count
0,Hipercor,1675728984,hipercor,2106,55496


In [18]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file, index=False)

saveID(df_account_info,'account_info_cabify.csv')
#saveID(dataFollowers_id,'followers_id.csv')

# Extracción de las cuentas Followers

In [44]:
#Nueva forma de obtener todas las cuentas que siguen a Uber/Cabify en España
#cabify_espana
#@Uber_ES
#NokiaMobile
# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_followers_ids(api, twitter_account='@hipercor'):
    while True:
        try:
            for page in tweepy.Cursor(api.followers_ids, id=twitter_account, count=5000).pages():
                followers_ids.extend(page)
                #print(len(followers_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return followers_ids

followers_ids=[]
get_followers_ids(api)
#Comprobación de que el número de followers es el correcto
print(len(followers_ids))


Rate limit reached. Sleeping for: 883


55496


In [45]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowers_id= pd.DataFrame(followers_ids)
dataFollowers_id.columns = ['id_Follower']
dataFollowers_id

,id_Follower
0,1243158426953359362
1,960961722
2,1451481704
3,471358027
4,375671507
...,...
55491,310844322
55492,824010588
55493,150045359
55494,559692939


In [46]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file, index=False)

saveID(dataFollowers_id,'followers_id_hipercor.csv')

# Extracción de las cuentas Followings

In [47]:
#Nueva forma de obtener todas las cuentas sigue El Corte Inglés/Cabify
#cabify_espana
#elcorteingles

# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@hipercor'):
    while True:
        try:
            for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
                following_ids.extend(page)
                #print(len(following_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return following_ids

following_ids=[]
get_following_ids(api)
print(len(following_ids))


2106


In [48]:
#Guardado de los ids en los ficheros para agilizar el desarrollo
dataFollowing_id= pd.DataFrame(following_ids)
dataFollowing_id.columns = ['id_Following']
dataFollowing_id


,id_Following
0,307366580
1,3149275991
2,955174650157457408
3,1019686070458441730
4,4728688119
...,...
2101,853023602
2102,30914064
2103,540750557
2104,157076499


In [49]:
#Guardado en ficheros de los IDS de Twitter
def saveID(dataframe,file):
    dataframe.to_csv(file, index=False)

saveID(dataFollowing_id,'followings_id_hipercor.csv')
#saveID(dataFollowers_id,'followers_id.csv')

# Definición de tramos de las cuentas Followers

In [50]:
#Agrupación de ID por la limitación de la API de Twitter
followers_limit=[followers_ids[n:n+99] for n in range(0, len(followers_ids), 100)]
print(len(followers_limit))

555


In [51]:
print(followers_limit)

[[1243158426953359362, 960961722, 1451481704, 471358027, 375671507, 1207650070096699398, 214577424, 1442327359, 981502477357993984, 294016462, 40081543, 130109540, 2399633241, 405250934, 610474213, 1228028220458635264, 1106557575405936641, 928612466, 1239168902711648257, 381589731, 1243831911124000768, 556025245, 248404580, 14697311, 278724064, 1142374813, 2989620797, 2965869647, 329813462, 547761674, 870841702231281664, 430594634, 2517942236, 4693148870, 1955353044, 984888558241636352, 108631464, 1631066419, 742256153904680960, 814913121424986112, 3122362006, 2939335486, 3345939453, 787008439872524288, 1231323902988759045, 1216404565496672263, 278413189, 1112836288930082816, 523649631, 1242922060373991430, 80970398, 2815742689, 1087749291790225408, 1481678084, 1172985931923636224, 983452437141970944, 1059078520368967681, 1239322339273293827, 199788094, 1243925049708630016, 854687980744736769, 2949612790, 101509056, 1178579253153189888, 428291695, 239855366, 216058802, 193212367, 73953

# Definición de tramos de las cuentas Following

In [52]:
#Agrupación de ID por la limitación de la API de Twitter
following_limit=[following_ids[n:n+99] for n in range(0, len(following_ids), 100)]
print(len(following_limit))

22


# Extracción de la información de las cuentas Followers

In [53]:
# Existe limitación al traer la información de los seguidores
data = []
for followers in followers_limit:
    follower_obj=api.lookup_users(user_ids=followers) 
    for user in follower_obj:
        follower={}
        follower["id"]=user.id
        follower["name"]=user.name
        follower["screen_name"]=user.screen_name
        follower["created_at"]=user.created_at
        follower["location"]=user.location
        follower["followers_count"]=user.followers_count
        follower["friends_count"]=user.friends_count
        follower["description"]=user.description
        data.append(follower)

df_followers_info = pd.DataFrame(data)
df_followers_info

,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,1243158426953359362,Juan Manuel Calzada Moreno,JMCalzamoreno,2020-03-26 12:51:22,"Malaga, Spain",1,57,Seguir viviendo
1,960961722,Dobra,GemaRucandio,2012-11-20 18:31:04,,32,359,
2,1451481704,AZ,anzame_,2013-05-23 13:18:49,,7,501,
3,471358027,Tony Hormaechea,TonyHormaechea,2012-01-22 19:36:18,,36,227,
4,375671507,Anxo,peterpanvive,2011-09-18 14:55:50,"Barcelona, Cataluña",93,205,9/6/2015
...,...,...,...,...,...,...,...,...
54937,310844322,David León,Abantwitt,2011-06-04 13:06:00,España,529,1506,Creativo gráfico Freelance y Networker\nhttps:...
54938,824010588,ralip,PilarPlopezg7,2012-09-14 21:19:27,,5,56,
54939,150045359,alberto ordoñez,alberto_odlr,2010-05-30 23:09:08,"Madrid, España",291,543,"Apasionado de la Comunicación y el Marketing, ..."
54940,559692939,mangriton,mangriton,2012-04-21 14:52:30,,28,142,


# Extracción información de las cuentas Followings

In [54]:
# Existe limitación al traer la información de los seguidores
data = []
for followings in following_limit:
    following_obj=api.lookup_users(user_ids=followings) 
    for user in following_obj:
        following={}
        following["id"]=user.id
        following["name"]=user.name
        following["screen_name"]=user.screen_name
        following["created_at"]=user.created_at
        following["location"]=user.location
        following["followers_count"]=user.followers_count
        following["friends_count"]=user.friends_count
        following["description"]=user.description
        data.append(following)

df_following_info = pd.DataFrame(data)
df_following_info

,id,name,screen_name,created_at,location,followers_count,friends_count,description
0,307366580,Leticia Blanquer,LetiBlanquer,2011-05-29 14:19:42,,84,169,
1,3149275991,Jorge Delgado,magustre,2015-04-08 20:39:35,,6,22,
2,955174650157457408,La del cesto,ladelcesto,2018-01-21 20:25:57,Galicia,43,237,Con una cesta llenita de opiniones para compar...
3,1019686070458441730,Veganoporaccidentespain,veganospain,2018-07-18 20:51:18,Spain,1117,9,"Productos de supermercados aptxs para veganxs,..."
4,4728688119,Antonio Jose Segovia,ajosesegovia,2016-01-06 22:29:14,Europe,161,384,"ISO 27001, ISO 22301, ISO 20000, ISO 27018, TI..."
...,...,...,...,...,...,...,...,...
2080,853023602,Receta y Cocina,RecetaYCocina,2012-09-29 14:53:20,,64212,37857,Recetas diarias y trucos de cocina para que pr...
2081,30914064,Tony Aguilar,TonyAguilarOfi,2009-04-13 17:50:48,Cornellà-Madrid,342330,4819,"Padre de Biel y Nil, Amante de La Radio, La Mú..."
2082,540750557,Banana!!,MonoComeBanana,2012-03-30 11:44:29,Valencia/Madrid,1763,1556,Soy Nutricionista y cocinillas. Me chifla ir a...
2083,157076499,SaborGourmet.com,saborgourmet,2010-06-18 20:04:52,Madrid,3434,1927,Las mejores #recetas de #cocina para degustar ...


In [55]:
#Guardado en ficheros de los IDS de Twitter
def saveInfo(dataframe,file):
    dataframe.to_csv(file, index=False)

saveInfo(df_following_info,'followings_info_hipercor.csv')
saveInfo(df_followers_info,'followers_info_hipercor.csv')


# -------------------------No tiene sentido de momento-------------------------------

# Extracción de la información de los Tweets de Followers

In [ ]:
#Extracción de los tweets de las cuentas de followers
data = []
for users in df_followers_info["screen_name"]:
    formato_user=str(users).replace(' ','')
    try:   
        tweets_user=api.user_timeline(screen_name=formato_user, count=1)
        for tweet in tweets_user:
            #Creación de diccionarios
            tweets={}
            tweets["id_tweet"]=tweet.id
            tweets["id_user"]=tweet.user.id
            tweets["text"]=tweet.text
            tweets["location"] = tweet.user.location
            tweets["created_at"]=tweet.created_at
            data.append(tweets)
    except tweepy.TweepError:
        print("Fallo por el usuario, ",tweet.user.id," Skipping...")
        pass
            
df_followers_info_tweets = pd.DataFrame(data)
df_followers_info_tweets

Fallo por el usuario,  85946106  Skipping...
Fallo por el usuario,  492595512  Skipping...
Fallo por el usuario,  1105167868621139968  Skipping...
Fallo por el usuario,  860100474715951104  Skipping...
Fallo por el usuario,  534771080  Skipping...
Fallo por el usuario,  1219916472086077440  Skipping...
Fallo por el usuario,  1076851198408491008  Skipping...
Fallo por el usuario,  1164276844398555136  Skipping...
Fallo por el usuario,  978689133009887232  Skipping...
Fallo por el usuario,  978689133009887232  Skipping...
Fallo por el usuario,  716574310236872704  Skipping...
Fallo por el usuario,  1221490818491539456  Skipping...
Fallo por el usuario,  77385383  Skipping...
Fallo por el usuario,  300416495  Skipping...
Fallo por el usuario,  3359130580  Skipping...
Fallo por el usuario,  1146332994346586112  Skipping...
Fallo por el usuario,  1068642743642177536  Skipping...
Fallo por el usuario,  1219382874459312130  Skipping...
Fallo por el usuario,  157800367  Skipping...
Fallo por e

Rate limit reached. Sleeping for: 551


Fallo por el usuario,  2473700540  Skipping...
Fallo por el usuario,  948094614447296512  Skipping...
Fallo por el usuario,  222880372  Skipping...
Fallo por el usuario,  47808922  Skipping...
Fallo por el usuario,  1349452261  Skipping...
Fallo por el usuario,  2550165192  Skipping...
Fallo por el usuario,  2550165192  Skipping...
Fallo por el usuario,  148545917  Skipping...
Fallo por el usuario,  3825432785  Skipping...
Fallo por el usuario,  175769602  Skipping...
Fallo por el usuario,  175769602  Skipping...
Fallo por el usuario,  940194606804819969  Skipping...
Fallo por el usuario,  16866767  Skipping...
Fallo por el usuario,  63854034  Skipping...
Fallo por el usuario,  490649122  Skipping...
Fallo por el usuario,  1206602409058979840  Skipping...
Fallo por el usuario,  361930079  Skipping...
Fallo por el usuario,  122479094  Skipping...
Fallo por el usuario,  743633306  Skipping...
Fallo por el usuario,  289014735  Skipping...
Fallo por el usuario,  407348888  Skipping...
Fall

Rate limit reached. Sleeping for: 570


Fallo por el usuario,  80122262  Skipping...
Fallo por el usuario,  1110541048424656897  Skipping...
Fallo por el usuario,  398076160  Skipping...
Fallo por el usuario,  1105580492663984129  Skipping...
Fallo por el usuario,  1105580492663984129  Skipping...
Fallo por el usuario,  1158390396688510976  Skipping...
Fallo por el usuario,  1159808156526141440  Skipping...
Fallo por el usuario,  1143911036896129029  Skipping...
Fallo por el usuario,  1143911036896129029  Skipping...
Fallo por el usuario,  264448270  Skipping...
Fallo por el usuario,  583920940  Skipping...
Fallo por el usuario,  2651450103  Skipping...
Fallo por el usuario,  1156806374132764672  Skipping...
Fallo por el usuario,  403012226  Skipping...
Fallo por el usuario,  47756149  Skipping...
Fallo por el usuario,  563864182  Skipping...
Fallo por el usuario,  468258123  Skipping...
Fallo por el usuario,  2166636186  Skipping...
Fallo por el usuario,  1319944850  Skipping...
Fallo por el usuario,  2600516221  Skipping..

...

Rate limit reached. Sleeping for: 579


Fallo por el usuario,  373075356  Skipping...
Fallo por el usuario,  45088970  Skipping...
Fallo por el usuario,  239378243  Skipping...
Fallo por el usuario,  563194546  Skipping...
Fallo por el usuario,  563194546  Skipping...
Fallo por el usuario,  137450386  Skipping...
Fallo por el usuario,  1003533942283296768  Skipping...
Fallo por el usuario,  416636571  Skipping...
Fallo por el usuario,  416636571  Skipping...
Fallo por el usuario,  2155044775  Skipping...
Fallo por el usuario,  2155044775  Skipping...
Fallo por el usuario,  2155044775  Skipping...
Fallo por el usuario,  164809876  Skipping...
Fallo por el usuario,  298420061  Skipping...
Fallo por el usuario,  8203462  Skipping...
Fallo por el usuario,  8203462  Skipping...
Fallo por el usuario,  411304022  Skipping...
Fallo por el usuario,  242962661  Skipping...
Fallo por el usuario,  557109228  Skipping...
Fallo por el usuario,  557109228  Skipping...
Fallo por el usuario,  987316429044232192  Skipping...
Fallo por el usua

Rate limit reached. Sleeping for: 572


Fallo por el usuario,  472327972  Skipping...
Fallo por el usuario,  126449833  Skipping...
Fallo por el usuario,  61569399  Skipping...
Fallo por el usuario,  375340011  Skipping...
Fallo por el usuario,  139000249  Skipping...
Fallo por el usuario,  139000249  Skipping...
Fallo por el usuario,  906710576  Skipping...
Fallo por el usuario,  353371761  Skipping...
Fallo por el usuario,  389397636  Skipping...
Fallo por el usuario,  1431594596  Skipping...
Fallo por el usuario,  56730456  Skipping...
Fallo por el usuario,  220958088  Skipping...
Fallo por el usuario,  99908204  Skipping...
Fallo por el usuario,  210132620  Skipping...
Fallo por el usuario,  734092114410262533  Skipping...
Fallo por el usuario,  2821116531  Skipping...
Fallo por el usuario,  993251414527938562  Skipping...
Fallo por el usuario,  1153668811  Skipping...
Fallo por el usuario,  287759778  Skipping...
Fallo por el usuario,  166160529  Skipping...
Fallo por el usuario,  166160529  Skipping...
Fallo por el usu

Rate limit reached. Sleeping for: 587


# Extracción de la información de los Tweets de Followings

In [22]:
#Extracción de los tweets de las cuentas de followers
data = []
for users in df_following_info["screen_name"]:
    formato_user=str(users).replace(' ','')
    try:   
        tweets_user=api.user_timeline(screen_name=formato_user, count=1)
        for tweet in tweets_user:
            #Creación de diccionarios
            tweets={}
            tweets["id_tweet"]=tweet.id
            tweets["id_user"]=tweet.user.id
            tweets["text"]=tweet.text
            tweets["location"] = tweet.user.location
            tweets["created_at"]=tweet.created_at
            data.append(tweets)
    except tweepy.TweepError:
        print("Fallo por el usuario, ",tweet.user.id," Skipping...")
        pass
            
df_following_info_tweets = pd.DataFrame(data)
df_following_info_tweets

Fallo por el usuario,  867615151951925248  Skipping...
Fallo por el usuario,  105895230  Skipping...
Fallo por el usuario,  816178  Skipping...
Fallo por el usuario,  169411324  Skipping...
Fallo por el usuario,  715656119839023104  Skipping...
Fallo por el usuario,  51736879  Skipping...
Fallo por el usuario,  145994453  Skipping...
Fallo por el usuario,  157705342  Skipping...
Fallo por el usuario,  316695810  Skipping...


,id_tweet,id_user,text,location,created_at
0,1225785867832381441,48422383,Some projects are about #accessibility 🙌 and I...,Transitando por el 🌍,2020-02-07 14:18:19
1,1225031619653795840,2992924523,Financial Times @FT entrevista a nuestro funda...,Global,2020-02-05 12:21:12
2,1225765733013098502,715535722086801408,RT @interbrandMAD: .@la_Mutua se mueve constan...,,2020-02-07 12:58:19
3,1222537192947535872,906103502841290752,RT @MadridSummitMS: ¡Esto es “Empresas por la ...,"Madrid, Comunidad de Madrid",2020-01-29 15:09:15
4,1225826091295813632,428136372,RT @martarias: Mañana sábado hay nueva edición...,"Barcelona, España",2020-02-07 16:58:09
5,1225816831367286784,913184120100081664,🤔 ¿Qué podemos hacer para tener un consumo de ...,"Madrid, España",2020-02-07 16:21:21
6,1225747481293709312,1076052488921460736,La nueva iniciativa en #movilidad sostenible e...,,2020-02-07 11:45:47
7,1225813993878671360,2848321193,"🗣️ @jdeelizalde: ""Tener muy claro el objetivo ...","Madrid, España",2020-02-07 16:10:05
8,1225799564235546624,9974322,📢Debería ser noticia 👇\n\nEl primer ensayo de ...,Madrid,2020-02-07 15:12:45
9,1225783557622243328,85343435,Tiempo de descanso al sol\n☀️\n☀️\n#estudiante...,#MuyCerca,2020-02-07 14:09:08


In [35]:
#Guardado en ficheros de los tweets por usuario de Twitter
def saveTweets(dataframe,file):
    dataframe.to_csv(file)

saveTweets(df_following_info_tweets,'followings_info_tweets.csv')
saveTweets(df_followers_info_tweets,'df_followers_info_tweets.csv')

NameError: name 'df_following_info_tweets' is not defined